<div style="background-color:#000;"><img src="pqn.png"></img></div>

We use math for numerical methods, matplotlib.pyplot for charting, numpy for numerical arrays, statsmodels and the regression module for statistical modeling, and yfinance for downloading market price data.

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import yfinance as yf
from statsmodels import regression

## Download and prepare historical data

This block pulls daily price data for the XLY ETF from Yahoo Finance using yfinance, keeping only the close prices for a 10-year period.

In [ ]:
close = yf.download(
    "XLY", period="10y", interval="1d", auto_adjust=False, progress=False
).Close
dates = close.index

The code here requests historical data for the XLY ETF, which tracks consumer discretionary stocks. We use the yfinance library to grab the data and focus only on the closing values, dropping everything else. The result is an indexed time series we'll analyze further.

## Run a log-linear regression

This block defines a function to fit a log-linear regression to the price data, modeling price growth as exponential over time.

In [ ]:
def loglinreg(X, Y):
    x = sm.add_constant(X)
    model = regression.linear_model.OLS(np.log(Y), x).fit()
    a, b = model.params

    X2 = np.linspace(X.min(), X.max(), 100)
    Y_hat = np.exp(a + b * X2)
    return model, X2, Y_hat

This function uses statsmodels to perform a regression where we assume the logarithm of the price changes linearly over time. It returns the regression model along with a smooth set of points for plotting. We use the exponential function to transform predicted values back to actual prices, so the output matches the original price scale.

This block generates our independent and dependent variables, fits the regression, and computes the fitted values.

In [ ]:
X = np.arange(len(close))
Y = close.values
model, X2, Y_hat = loglinreg(X, Y)

Here, we assign each time point a sequential index number for regression. We send these numbers and the corresponding price values into our log-linear regression. The model outputs both the statistical fit and a set of predicted prices, ready for charting.

## Visualize the regression fit

This block creates a chart showing both the actual XLY price data and the fitted exponential growth curve.

In [ ]:
fig, ax = plt.subplots()
ax.plot(dates, Y, label="XLY Price")
ax.plot(dates[X2.astype(int)], Y_hat, "r", label="Log-Linear Fit")

# Formatting
ax.set_title("XLY Price with Log-Linear Regression Fit")
ax.set_xlabel("Date")
ax.set_ylabel("Price")
ax.legend()

# Auto-handle x-axis ticks/dates
fig.autofmt_xdate()
plt.show()

We use matplotlib to chart both the real ETF price (in blue) and our fitted exponential growth path (in red), plotting them on the same date axis. The chart clearly shows how closely our regression follows the general trend in price over the decade. We label each line and axis, then adjust the dates so the chart is legible. This helps us see at a glance how well our model describes the price movement.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.